**Transforming Raw Data into Actionable Business Insights**

In [ ]:
import sys
!pip install numpy
!pip install plotly
!pip install datapane

In [ ]:
from pyspark.sql import SparkSession
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import warnings
import sys
import os

# Start SparkSession
spark = SparkSession.builder.appName("EDM_Final_Project").getOrCreate()

warnings.filterwarnings('ignore')

In [ ]:
# JDBC Connection in PySpark
jdbc_url = "jdbc:mysql://database-1.c1r9lsqj9mxc.us-east-1.rds.amazonaws.com/gdb041"
connection_properties = {
    "user": "admin",
    "password": "Arbaz786$",
    "driver": "com.mysql.jdbc.Driver"
}

# Check connection by reading a small table
version_df = spark.read.jdbc(url=jdbc_url, table="(SELECT VERSION() AS version) as ver", properties=connection_properties)
version_df.show()

Database version: 8.0.33 


In [ ]:
def run_query(query: str):
    return spark.read.jdbc(url=jdbc_url, table=f"({query}) as tmp", properties=connection_properties)

def run_command(command: str):
    # In PySpark, you generally do not need to run commands like in SQL, but here's an example:
    spark.sql(command)

In [ ]:
def show_tables():
    query = """
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'gdb041' AND table_type = 'BASE TABLE';
    """
    return run_query(query)

def get_table_row_count(tablename: str):
    query = f"SELECT COUNT(1) FROM {tablename};"
    df = run_query(query)
    return df.collect()[0][0]

tables = show_tables()
tables = tables.withColumn("row_count", [get_table_row_count(t['table_name']) for t in tables.collect()])
tables.show()

,TABLE_NAME,row_count
0,dim_customer,209
1,dim_market,27
2,dim_product,397
3,fact_forecast_monthly,1885941
4,fact_sales_monthly,1425706
5,freight_cost,135
6,gross_price,1197
7,manufacturing_cost,1197
8,post_invoice_deductions,2063076
9,pre_invoice_deductions,1045


# Building a dashboard

Bar Chart - Total Sales by Product Category:

In [ ]:

Total_Sales_by_Product_Category_query = """
SELECT category,
       SUM(sold_quantity) AS total_sold_quantity,
       SUM(sold_quantity * gross_price) AS total_revenue
FROM gdb041.fact_sales_monthly F
JOIN gdb041.gross_price G ON F.product_code = G.product_code
AND YEAR(F.date) = G.fiscal_year
GROUP BY category;
"""

total_sales_df = spark.sql(Total_Sales_by_Product_Category_query)

total_sales_df.show()

In [ ]:
total_sales_plot = px.bar(total_sales_df, x='category', y='total_revenue', color = 'category', title='Total Sales by Product Category')
total_sales_plot.show()

Line Chart - Monthly Sales Trends for a Specific Product:

In [ ]:
Monthly_Sales_Trends_for_a_Specific_Product_query="""
SELECT date, SUM(sold_quantity) AS monthly_sales
FROM fact_sales_monthly
WHERE product_code = 'A0418150101'
GROUP BY date
ORDER BY date;
"""
sales_trends_df=run_spark.sqlquery(Monthly_Sales_Trends_for_a_Specific_Product_query)
sales_trends_df.show()

In [ ]:
monthly_tales_trends = px.line(sales_trends_df, x='date', y='monthly_sales', title='Monthly Sales Trends for a Specific Product')
monthly_tales_trends.show()

Pie Chart - Market Share of Top Products:

In [ ]:
Market_Share_of_Top_Products_query="""
WITH top_products AS (
  SELECT product_code, SUM(sold_quantity) AS total_sold
  FROM fact_sales_monthly
  GROUP BY product_code
  ORDER BY total_sold DESC
  LIMIT 5
)
SELECT tp.product_code, tp.total_sold, SUM(total_sold) OVER () AS total_sales
FROM top_products tp;
"""
market_share_df=spark.sql(Market_Share_of_Top_Products_query)
market_share_df.show()

In [ ]:
market_share = px.pie(market_share_df, names='product_code', values='total_sold', title='Market Share of Top Products')
market_share.show()

Pie chart - Top 3 Customers by Purchases:

In [ ]:
Top_3_Customers_by_Purchases_query="""
SELECT customer_code, customer_name, SUM(sold_quantity) AS total_purchases
FROM gdb041.fact_sales_monthly
WHERE YEAR(date) = YEAR('2020-01-01 00:00:00')
GROUP BY customer_code, customer_name
ORDER BY total_purchases DESC
LIMIT 3;
"""

top_purchases_df=spark.sql(Top_3_Customers_by_Purchases_query)

top_purchases_df.show()

In [ ]:
top_3_purchases = px.pie(top_purchases_df, names='customer_name', values='total_purchases', title='Top 3 Customers by Purchases')
top_3_purchases.show()

Time Series - Sales Trends Over Time:

In [ ]:
Sales_Trends_Over_Time_query="""
SELECT date, SUM(sold_quantity) AS total_quantity_sold
FROM fact_sales_monthly
GROUP BY date
ORDER BY date;
"""

sales_df=spark.sql(Sales_Trends_Over_Time_query)

sales_df.show()

In [ ]:
sales_trend = px.line(sales_df, x='date', y='total_quantity_sold', title='Sales Trends Over Time')
sales_trend.show()

Bar Plot - Total Number of Products sold by Product Category:

In [ ]:
Total_Number_of_Products_sold_by_Product_Category_query="""
SELECT category,
       SUM(sold_quantity) AS total_sold_quantity,
       SUM(sold_quantity * gross_price) AS total_revenue
FROM gdb041.fact_sales_monthly F
JOIN gdb041.gross_price G ON F.product_code = G.product_code
AND YEAR(F.date) = G.fiscal_year
GROUP BY category;
"""

total_df=spark.sql(Total_Number_of_Products_sold_by_Product_Category_query)
total_df.show()

In [ ]:
total_sold_plot = px.bar(total_df, x='category', y='total_sold_quantity', color = 'category', title='Distribution of Manufacturing Costs by Product')
total_sold_plot.show()

In [ ]:
import altair as alt
import datapane as dp
dp.enable_logging()

Putting it into a Datapane report

In [ ]:
customers = spark.sql("SELECT COUNT(*) AS row_count FROM dim_customer").collect()[0]['row_count']
markets = spark.sql("SELECT COUNT(*) AS row_count FROM dim_market").collect()[0]['row_count']
products = spark.sql("SELECT COUNT(*) AS row_count FROM dim_product").collect()[0]['row_count']
post_invoice_deductions = spark.sql("SELECT COUNT(*) AS row_count FROM post_invoice_deductions").collect()[0]['row_count']

In [ ]:
# Create the Datapane report with the visualizations and data
r = dp.View(
    dp.Group(
        dp.BigNumber(heading="Total customers", value=customers),
        dp.BigNumber(heading="Total markets", value=markets),
        dp.BigNumber(heading="Total products sold", value=products),
        columns=3,
        name="Little_group",
    ),
    dp.Plot(total_sales_plot, name="total_sales"),           # Ensure this is a pandas DataFrame or Altair chart
    dp.Plot(monthly_tales_trends, name="monthly_tales_trends"), # Ensure this is a pandas DataFrame or Altair chart
    dp.Group(dp.Plot(market_share), dp.Plot(sales_trend), dp.Plot(top_3_purchases), dp.Plot(total_sold_plot), columns=2),
)

# Save the report as an HTML file
dp.save_report(r, "report.html", open=True)

[08:30:35] [DEBUG] <View version="1" fragment="false">
  <Group name="Little_group" columns="3" valign="top">
    <BigNumber heading="Total customers" value="209" is_positive_intent="false" is_upward_change="false"/>
    <BigNumber heading="Total markets" value="27" is_positive_intent="false" is_upward_change="false"/>
    <BigNumber heading="Total products sold" value="397" is_positive_intent="false" is_upward_change="false"/>
  </Group>
  <Plot type="application/vnd.plotly.v1+json" name="total_sales" responsive="true" scale="1.0" src="ref://327e8c233a"/>
  <Plot type="application/vnd.plotly.v1+json" name="monthly_tales_trends" responsive="true" scale="1.0" src="ref://a7e457a162"/>
  <Group columns="2" valign="top">
    <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://e90514c238"/>
    <Plot type="application/vnd.plotly.v1+json" responsive="true" scale="1.0" src="ref://09ebbb4dda"/>
    <Plot type="application/vnd.plotly.v1+json" responsive="true" s

App saved to ./report.html

In [ ]:
import IPython
IPython.display.HTML(filename='report.html')

In [ ]:
from google.colab import files

# Download the HTML file
files.download('report.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>